# Medical Exam Recommendation Based on Symptoms

Grid Search with:
- Decision Tree Classifier
- Random Forest Classifier
- KNN
- Nayve Bayes
- SVM

In [ ]:
import numpy as np
import pandas as pd
import pickle

from sklearn import ensemble
from sklearn import multioutput
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn import naive_bayes
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.insert(0, '/content/drive/My Drive/LEI')

In [ ]:
!unzip -q '/content/drive/My Drive/LEI/Data'

#### Load Symptom Dataset

In [ ]:
data = pd.read_csv('./Data/final.csv', sep = ';')

In [ ]:
data

,Blood Tests,Skin biopsy,Patch test,Complete blood count (CBC),Polymerase chain reaction (PCR),Skin Prick Test (SPT),Intradermal Skin Test,Physical Examination,TSH test,T4 test,Thyroid scan,Thyroid ultrasound,Esophagram,Esophageal manometry,pH monitoring,Endoscopy,Biopsy of upper disgestive system,X-ray of upper digestive system,Serum bilirubin test,Serum albumin test,Serum alkaline phosphatase test,Serum aminotransferases (transaminases),Prothrombin time (PTT) test,Alanine transaminase (ALT) test,Liver Ultrasound,Liver Biopsy,MRI Scan,CT Scan,X-ray,Electronystagmography (ENG),Videonystagmography (VNG),Fasting plasma glucose (FPG) test,Hemoglobin A1C test,Random plasma glucose (RPG) test,Blood Test,Electrocardiogram (ECG),Echocardiogram,Ambulatory monitoring,Urine analysis,Upper gastrointestinal endoscopy,...,Bloody Stool,irritationinanus,Cramps,Bruising,Swollen Legs,Swollen Blood Vessels,Prominent Veins on Calf,Weight Gain,Cold Hands and Feet,Mood Swings,Puffy Face and Eyes,Enlarged Thyroid,Brittle Nails,Swollen Extremeties,Abnormal Menstruation,Muscle Weakness,Anxiety,Slurred Speech,Palpitations,Drying and Tingling Lips,Knee Pain,Hip Joint Pain,Swelling Joints,Painful Walking,Movement Stiffness,Spinning Movements,Unsteadiness,Pus Filled Pimples,Blackheads,Scurring,Bladder Discomfort,Foul Smell of Urine,Continuous Feel of Urine,Skin Peeling,Silver Like Dusting,Small Dents in Nails,Inflammatory Nails,Blister,Red Sore Around Nose,Yellow Crust Ooze
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4675,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0
4676,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,0,0,0,0,0,0,0,0,0,0
4677,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0
4678,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0


####  Load Symptom Severity 

In [ ]:
severity = pd.read_csv('./Data/Symptom-severity.csv')

####  Load Exam Mapping

In [ ]:
mapping = pd.read_csv('./Data/map.csv', sep = ';')

# Split data for train and test

In [ ]:
symptomCount = 131
testCount = 102

[train,test] = train_test_split(data,random_state=420)
trainA = np.array(train)
trainAX = trainA[:,testCount:]
trainAY = trainA[:,:testCount]

trainX=train.iloc[:,testCount:]
trainY=train.iloc[:,:testCount]

testX=test.iloc[:,testCount:]
testY=test.iloc[:,:testCount]

# Grid Search

In [ ]:
def realAccuracies(predictions, labels):
    thresh =(predictions.max()+ predictions.min())/2
    predictions[predictions>thresh]=1
    predictions[predictions<=thresh]=0
    
    correctByLine=[]
    for i in range(len(testY)):
        matches = predictions[i]==labels[i]
        correctByLine.append(len(matches[matches==True])/len(matches))
        
    return([len(np.nonzero([x for x in correctByLine if x==1])[0]),
            len(np.nonzero([x for x in correctByLine if x>0.99])[0]),
            len(np.nonzero([x for x in correctByLine if x>0.95])[0]),
            len(np.nonzero([x for x in correctByLine if x>0.90])[0]),
            len(np.nonzero([x for x in correctByLine if x!=1])[0])])


def getRealAcc(clf):
    predicted = clf.predict(testX)
    accs =  realAccuracies(predicted, testY.to_numpy())
    print('')
    print(accs[3]," classified 90% correctly")
    print(accs[2]," classified 95% correctly")
    print(accs[1]," classified 99% correctly")
    print(accs[0]," classified 100% correctly")
    print(accs[4]," classified incorrectly (Not 100%)")
    return predicted

In [ ]:
def gridSearch(model, param_grid, X_train, y_train, X_test, y_test):        
    clf = GridSearchCV(model, param_grid, refit=True, verbose=3)
    clf.fit(X_train,y_train)
    print(clf.best_params_)
    predictions = clf.predict(testX)
    predicted = getRealAcc(clf)
    return ;

# Random Forest Classifier

In [ ]:
grid_params_randomForest = {
    'n_estimators' : [10,20,30,50,100,200,1000],
    'max_depth' : [1, 10, 20, None],
    'bootstrap': [True, False],
}

gridSearch(RandomForestClassifier(), grid_params_randomForest, trainX, trainY, testX, testY)


Fitting 5 folds for each of 56 candidates, totalling 280 fits
[CV] bootstrap=True, max_depth=1, n_estimators=10 ....................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  bootstrap=True, max_depth=1, n_estimators=10, score=0.027, total=   0.3s
[CV] bootstrap=True, max_depth=1, n_estimators=10 ....................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV]  bootstrap=True, max_depth=1, n_estimators=10, score=0.021, total=   0.3s
[CV] bootstrap=True, max_depth=1, n_estimators=10 ....................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.6s remaining:    0.0s


[CV]  bootstrap=True, max_depth=1, n_estimators=10, score=0.017, total=   0.3s
[CV] bootstrap=True, max_depth=1, n_estimators=10 ....................
[CV]  bootstrap=True, max_depth=1, n_estimators=10, score=0.028, total=   0.3s
[CV] bootstrap=True, max_depth=1, n_estimators=10 ....................
[CV]  bootstrap=True, max_depth=1, n_estimators=10, score=0.078, total=   0.3s
[CV] bootstrap=True, max_depth=1, n_estimators=20 ....................
[CV]  bootstrap=True, max_depth=1, n_estimators=20, score=0.027, total=   0.5s
[CV] bootstrap=True, max_depth=1, n_estimators=20 ....................
[CV]  bootstrap=True, max_depth=1, n_estimators=20, score=0.021, total=   0.5s
[CV] bootstrap=True, max_depth=1, n_estimators=20 ....................
[CV]  bootstrap=True, max_depth=1, n_estimators=20, score=0.017, total=   0.5s
[CV] bootstrap=True, max_depth=1, n_estimators=20 ....................
[CV]  bootstrap=True, max_depth=1, n_estimators=20, score=0.028, total=   0.5s
[CV] bootstrap=True, 

[Parallel(n_jobs=1)]: Done 280 out of 280 | elapsed: 26.3min finished


{'bootstrap': True, 'max_depth': None, 'n_estimators': 20}

1170  classified 90% correctly
1170  classified 95% correctly
1170  classified 99% correctly
1170  classified 100% correctly
0  classified incorrectly (Not 100%)


# Decision Tree Classifier

In [ ]:
grid_params_decisionTree = {
    'criterion' : ['gini', 'entropy'],
    'splitter' : ['best', 'random'],
    'max_depth' : [1, 10, 20, None],
}

gridSearch(DecisionTreeClassifier(), grid_params_decisionTree, trainX, trainY, testX, testY)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] criterion=gini, max_depth=1, splitter=best ......................
[CV]  criterion=gini, max_depth=1, splitter=best, score=0.060, total=   0.0s
[CV] criterion=gini, max_depth=1, splitter=best ......................
[CV]  criterion=gini, max_depth=1, splitter=best, score=0.037, total=   0.0s
[CV] criterion=gini, max_depth=1, splitter=best ......................
[CV]  criterion=gini, max_depth=1, splitter=best, score=0.044, total=   0.0s
[CV] criterion=gini, max_depth=1, splitter=best ......................
[CV]  criterion=gini, max_depth=1, splitter=best, score=0.053, total=   0.0s
[CV] criterion=gini, max_depth=1, splitter=best ......................
[CV]  criterion=gini, max_depth=1, splitter=best, score=0.074, total=   0.0s
[CV] criterion=gini, max_depth=1, splitter=random ....................
[CV]  criterion=gini, max_depth=1, splitter=random, score=0.060, total=   0.0s
[CV] criterion=gini, max_depth=1, splitter=random

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV]  criterion=gini, max_depth=1, splitter=random, score=0.044, total=   0.0s
[CV] criterion=gini, max_depth=1, splitter=random ....................
[CV]  criterion=gini, max_depth=1, splitter=random, score=0.053, total=   0.0s
[CV] criterion=gini, max_depth=1, splitter=random ....................
[CV]  criterion=gini, max_depth=1, splitter=random, score=0.074, total=   0.0s
[CV] criterion=gini, max_depth=10, splitter=best .....................
[CV]  criterion=gini, max_depth=10, splitter=best, score=0.509, total=   0.1s
[CV] criterion=gini, max_depth=10, splitter=best .....................
[CV]  criterion=gini, max_depth=10, splitter=best, score=0.620, total=   0.1s
[CV] criterion=gini, max_depth=10, splitter=best .....................
[CV]  criterion=gini, max_depth=10, splitter=best, score=0.623, total=   0.1s
[CV] criterion=gini, max_depth=10, splitter=best .....................
[CV]  criterion=gini, max_depth=10, splitter=best, score=0.566, total=   0.1s
[CV] criterion=gini, max_

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    4.8s finished


# KNN

In [ ]:
grid_params_knn = {
    'n_neighbors' : [3,5,7,11,13,15,17,25],
    'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'weights' : ['uniform', 'distance'],
    'metric' : ['euclidean', 'manhattan'],
}

gridSearch(KNeighborsClassifier(), grid_params_knn, trainX, trainY, testX, testY)

Fitting 5 folds for each of 128 candidates, totalling 640 fits
[CV] algorithm=auto, metric=euclidean, n_neighbors=3, weights=uniform 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  algorithm=auto, metric=euclidean, n_neighbors=3, weights=uniform, score=1.000, total=   2.0s
[CV] algorithm=auto, metric=euclidean, n_neighbors=3, weights=uniform 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s


[CV]  algorithm=auto, metric=euclidean, n_neighbors=3, weights=uniform, score=0.997, total=   1.9s
[CV] algorithm=auto, metric=euclidean, n_neighbors=3, weights=uniform 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.9s remaining:    0.0s


[CV]  algorithm=auto, metric=euclidean, n_neighbors=3, weights=uniform, score=1.000, total=   1.9s
[CV] algorithm=auto, metric=euclidean, n_neighbors=3, weights=uniform 
[CV]  algorithm=auto, metric=euclidean, n_neighbors=3, weights=uniform, score=1.000, total=   1.9s
[CV] algorithm=auto, metric=euclidean, n_neighbors=3, weights=uniform 
[CV]  algorithm=auto, metric=euclidean, n_neighbors=3, weights=uniform, score=1.000, total=   1.9s
[CV] algorithm=auto, metric=euclidean, n_neighbors=3, weights=distance 
[CV]  algorithm=auto, metric=euclidean, n_neighbors=3, weights=distance, score=1.000, total=   0.3s
[CV] algorithm=auto, metric=euclidean, n_neighbors=3, weights=distance 
[CV]  algorithm=auto, metric=euclidean, n_neighbors=3, weights=distance, score=1.000, total=   0.3s
[CV] algorithm=auto, metric=euclidean, n_neighbors=3, weights=distance 
[CV]  algorithm=auto, metric=euclidean, n_neighbors=3, weights=distance, score=1.000, total=   0.3s
[CV] algorithm=auto, metric=euclidean, n_neig

[Parallel(n_jobs=1)]: Done 640 out of 640 | elapsed: 11.5min finished



1170  classified 90% correctly
1170  classified 95% correctly
1170  classified 99% correctly
1170  classified 100% correctly
0  classified incorrectly (Not 100%)


# Nayve Bayes

In [ ]:
grid_params_gaussian = {
    'estimator__var_smoothing': [1e-11, 1e-10, 1e-9]
}

gaussianNB = multioutput.MultiOutputClassifier(naive_bayes.GaussianNB())
#print('Gaussian Params:',  gaussianNB.get_params().keys())

nSteps = 10
grid_params_bernoulli = {
    'estimator__alpha': np.array(range(nSteps))/nSteps,
    'estimator__fit_prior':[True,False]
}

bernoulli = multioutput.MultiOutputClassifier(naive_bayes.BernoulliNB())
#print('Bernoulli Params:', bernoulli.get_params().keys())

grid_params_multinominal = {
    'estimator__alpha': np.array(range(nSteps))/nSteps,
}

multinominal = multioutput.MultiOutputClassifier(naive_bayes.MultinomialNB())
#print('Multinominal Params:', multinominal.get_params().keys())

In [ ]:
gridSearch(gaussianNB, grid_params_gaussian, trainX, trainY, testX, testY)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] estimator__var_smoothing=1e-11 ..................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ...... estimator__var_smoothing=1e-11, score=0.859, total=   0.6s
[CV] estimator__var_smoothing=1e-11 ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV] ...... estimator__var_smoothing=1e-11, score=0.895, total=   0.6s
[CV] estimator__var_smoothing=1e-11 ..................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.2s remaining:    0.0s


[CV] ...... estimator__var_smoothing=1e-11, score=0.855, total=   0.6s
[CV] estimator__var_smoothing=1e-11 ..................................
[CV] ...... estimator__var_smoothing=1e-11, score=0.866, total=   0.6s
[CV] estimator__var_smoothing=1e-11 ..................................
[CV] ...... estimator__var_smoothing=1e-11, score=0.877, total=   0.6s
[CV] estimator__var_smoothing=1e-10 ..................................
[CV] ...... estimator__var_smoothing=1e-10, score=0.859, total=   0.6s
[CV] estimator__var_smoothing=1e-10 ..................................
[CV] ...... estimator__var_smoothing=1e-10, score=0.895, total=   0.6s
[CV] estimator__var_smoothing=1e-10 ..................................
[CV] ...... estimator__var_smoothing=1e-10, score=0.855, total=   0.6s
[CV] estimator__var_smoothing=1e-10 ..................................
[CV] ...... estimator__var_smoothing=1e-10, score=0.866, total=   0.6s
[CV] estimator__var_smoothing=1e-10 ..................................
[CV] .

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    9.0s finished


{'estimator__var_smoothing': 1e-11}

1170  classified 90% correctly
1142  classified 95% correctly
1076  classified 99% correctly
1011  classified 100% correctly
159  classified incorrectly (Not 100%)


In [ ]:
gridSearch(bernoulli, grid_params_bernoulli, trainX, trainY, testX, testY)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] estimator__alpha=0.0, estimator__fit_prior=True .................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  estimator__alpha=0.0, estimator__fit_prior=True, score=0.909, total=   0.6s
[CV] estimator__alpha=0.0, estimator__fit_prior=True .................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV]  estimator__alpha=0.0, estimator__fit_prior=True, score=0.944, total=   0.5s
[CV] estimator__alpha=0.0, estimator__fit_prior=True .................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.1s remaining:    0.0s


[CV]  estimator__alpha=0.0, estimator__fit_prior=True, score=0.910, total=   0.5s
[CV] estimator__alpha=0.0, estimator__fit_prior=True .................
[CV]  estimator__alpha=0.0, estimator__fit_prior=True, score=0.932, total=   0.5s
[CV] estimator__alpha=0.0, estimator__fit_prior=True .................
[CV]  estimator__alpha=0.0, estimator__fit_prior=True, score=0.925, total=   0.5s
[CV] estimator__alpha=0.0, estimator__fit_prior=False ................
[CV]  estimator__alpha=0.0, estimator__fit_prior=False, score=0.907, total=   0.5s
[CV] estimator__alpha=0.0, estimator__fit_prior=False ................
[CV]  estimator__alpha=0.0, estimator__fit_prior=False, score=0.939, total=   0.5s
[CV] estimator__alpha=0.0, estimator__fit_prior=False ................
[CV]  estimator__alpha=0.0, estimator__fit_prior=False, score=0.913, total=   0.5s
[CV] estimator__alpha=0.0, estimator__fit_prior=False ................
[CV]  estimator__alpha=0.0, estimator__fit_prior=False, score=0.929, total=   0

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   50.5s finished


{'estimator__alpha': 0.0, 'estimator__fit_prior': True}

1170  classified 90% correctly
1144  classified 95% correctly
1104  classified 99% correctly
1084  classified 100% correctly
86  classified incorrectly (Not 100%)


In [ ]:
gridSearch(multinominal, grid_params_multinominal, trainX, trainY, testX, testY)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] estimator__alpha=0.0 ............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................ estimator__alpha=0.0, score=0.905, total=   0.3s
[CV] estimator__alpha=0.0 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV] ................ estimator__alpha=0.0, score=0.940, total=   0.2s
[CV] estimator__alpha=0.0 ............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s


[CV] ................ estimator__alpha=0.0, score=0.905, total=   0.2s
[CV] estimator__alpha=0.0 ............................................
[CV] ................ estimator__alpha=0.0, score=0.927, total=   0.3s
[CV] estimator__alpha=0.0 ............................................
[CV] ................ estimator__alpha=0.0, score=0.919, total=   0.3s
[CV] estimator__alpha=0.1 ............................................
[CV] ................ estimator__alpha=0.1, score=0.811, total=   0.2s
[CV] estimator__alpha=0.1 ............................................
[CV] ................ estimator__alpha=0.1, score=0.829, total=   0.2s
[CV] estimator__alpha=0.1 ............................................
[CV] ................ estimator__alpha=0.1, score=0.792, total=   0.3s
[CV] estimator__alpha=0.1 ............................................
[CV] ................ estimator__alpha=0.1, score=0.771, total=   0.2s
[CV] estimator__alpha=0.1 ............................................
[CV] .

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   12.5s finished


{'estimator__alpha': 0.0}

1170  classified 90% correctly
1165  classified 95% correctly
1080  classified 99% correctly
1074  classified 100% correctly
96  classified incorrectly (Not 100%)


# SVM

In [ ]:
grid_params_svc = {
    'estimator__kernel': ['poly','rbf','linear'],
    'estimator__gamma': [1,0.1,0.01,0.001],
    'estimator__C': [0.1,1, 10, 100],
    'estimator__class_weight': ['balanced', None]
} 

In [ ]:
from sklearn.svm import LinearSVC
svc = OneVsRestClassifier(SVC()).fit(trainX.to_numpy(), trainY.to_numpy())
gridSearch(svc, grid_params_svc, trainX, trainY.to_numpy(), testX, testY)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] estimator__C=0.1, estimator__class_weight=balanced, estimator__gamma=1, estimator__kernel=poly 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  estimator__C=0.1, estimator__class_weight=balanced, estimator__gamma=1, estimator__kernel=poly, score=1.000, total=   4.1s
[CV] estimator__C=0.1, estimator__class_weight=balanced, estimator__gamma=1, estimator__kernel=poly 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.2s remaining:    0.0s


[CV]  estimator__C=0.1, estimator__class_weight=balanced, estimator__gamma=1, estimator__kernel=poly, score=1.000, total=   4.1s
[CV] estimator__C=0.1, estimator__class_weight=balanced, estimator__gamma=1, estimator__kernel=poly 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    8.3s remaining:    0.0s


[CV]  estimator__C=0.1, estimator__class_weight=balanced, estimator__gamma=1, estimator__kernel=poly, score=1.000, total=   4.1s
[CV] estimator__C=0.1, estimator__class_weight=balanced, estimator__gamma=1, estimator__kernel=poly 
[CV]  estimator__C=0.1, estimator__class_weight=balanced, estimator__gamma=1, estimator__kernel=poly, score=1.000, total=   4.1s
[CV] estimator__C=0.1, estimator__class_weight=balanced, estimator__gamma=1, estimator__kernel=poly 
[CV]  estimator__C=0.1, estimator__class_weight=balanced, estimator__gamma=1, estimator__kernel=poly, score=1.000, total=   4.1s
[CV] estimator__C=0.1, estimator__class_weight=balanced, estimator__gamma=1, estimator__kernel=rbf 
[CV]  estimator__C=0.1, estimator__class_weight=balanced, estimator__gamma=1, estimator__kernel=rbf, score=0.972, total=  27.5s
[CV] estimator__C=0.1, estimator__class_weight=balanced, estimator__gamma=1, estimator__kernel=rbf 
[CV]  estimator__C=0.1, estimator__class_weight=balanced, estimator__gamma=1, estim

[Parallel(n_jobs=1)]: Done 480 out of 480 | elapsed: 104.9min finished


{'estimator__C': 0.1, 'estimator__class_weight': 'balanced', 'estimator__gamma': 1, 'estimator__kernel': 'poly'}

1170  classified 90% correctly
1170  classified 95% correctly
1170  classified 99% correctly
1170  classified 100% correctly
0  classified incorrectly (Not 100%)


# Run and save the best Random Forest Tree

In [ ]:
model = RandomForestClassifier(bootstrap = True, max_depth= None, n_estimators = 20) 

model.fit(trainX, trainY)

# save the model to disk
filename = 'bestRFT.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(testX, testY)
print(result)

1.0


# Run and save the best Decicion Tree 

In [ ]:
model = DecisionTreeClassifier(criterion= 'entropy', max_depth = None, splitter= 'best') 

model.fit(trainX, trainY)

# save the model to disk
filename = 'bestDT.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(testX, testY)
print(result)

1.0


# Run and save the best KNN

In [ ]:
model = KNeighborsClassifier(algorithm = 'auto', metric='euclidean', n_neighbors = 3, weights = 'distance') 

model.fit(trainX, trainY)

# save the model to disk
filename = 'bestKnn.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(testX, testY)
print(result)

1.0
